In [1]:
import os
import datetime
import numpy as np
import cv2
import natsort
import math

In [2]:
#Determine depth

def getMAxChannel(img):
    imgGray = np.zeros((img.shape[0], img.shape[1]), 'float32')
    for i in range(0, img.shape[0]):
        for j in range(0, img.shape[1]):
            localMax = 0
            for k in range(0, 2):
                if img.item((i, j, k)) > localMax:
                    localMax = img.item((i, j, k))
            imgGray[i, j] = localMax
    return imgGray


def getDarkChannel(img, blockSize):
    addSize = int((blockSize - 1) / 2)
    newHeight = img.shape[0] + blockSize - 1
    newWidth = img.shape[1] + blockSize - 1

    imgMiddle = np.zeros((newHeight, newWidth))
    imgMiddle[:, :] = 0
    imgMiddle[addSize:newHeight - addSize, addSize:newWidth - addSize] = img
    # print('imgMiddle',imgMiddle)
    imgDark = np.zeros((img.shape[0], img.shape[1]), dtype=np.float16)
    localMax = 0
    for i in range(addSize, newHeight - addSize):
        for j in range(addSize, newWidth - addSize):
            localMax = 0
            for k in range(i - addSize, i + addSize + 1):
                for l in range(j - addSize, j + addSize + 1):
                    if imgMiddle.item((k, l)) > localMax:
                        localMax = imgMiddle.item((k, l))
            imgDark[i - addSize, j - addSize] = localMax
    return imgDark


def determineDepth(img, blockSize):
    img2 = img/255
    img_GB = getMAxChannel(img2)
    Max_GB = getDarkChannel(img_GB, blockSize)
    Max_R  = getDarkChannel(img2[:,:,2], blockSize)
    largestDiff = Max_R  - Max_GB

    return largestDiff

In [3]:
#Transmission Estimation

def getTransmission(normI,AtomsphericLight ,w):
    M, N, C = normI.shape #M are the rows, N are the columns, C is the bgr channel
    B = AtomsphericLight
    padwidth = math.floor(w/2)
    padded = np.pad(normI/B, ((padwidth, padwidth), (padwidth, padwidth),(0,0)), 'constant')
    transmission = np.zeros((M,N,2))
    for y, x in np.ndindex(M, N):
        transmission[y,x,0] = 1 - np.min(padded[y : y+w , x : x+w , 0])
        transmission[y,x,1] = 1 - np.min(padded[y : y+w , x : x+w , 1])
    return transmission

In [4]:
# Guided Filter

class GuidedFilter:
    
    # def __init__(self, I, radius=5, epsilon=0.4):
    def __init__(self, I, radius, epsilon):

        self._radius = 2 * radius + 1
        self._epsilon = epsilon
        self._I = self._toFloatImg(I)
        self._initFilter()

        # print('radius',self._radius)
        # print('epsilon',self._epsilon)

    def _toFloatImg(self, img):
        if img.dtype == np.float32:
            return img
        return ( 1.0 / 255.0 ) * np.float32(img)

    def _initFilter(self):
        I = self._I
        r = self._radius
        eps = self._epsilon

        Ir, Ig, Ib = I[:, :, 0], I[:, :, 1], I[:, :, 2]

        # self._Ir_mean = cv2.blur(Ir, (r, r))
        # self._Ig_mean = cv2.blur(Ig, (r, r))
        # self._Ib_mean = cv2.blur(Ib, (r, r))
        #
        # Irr_var = cv2.blur(Ir ** 2, (r, r)) - self._Ir_mean ** 2 + eps
        # Irg_var = cv2.blur(Ir * Ig, (r, r)) - self._Ir_mean * self._Ig_mean
        # Irb_var = cv2.blur(Ir * Ib, (r, r)) - self._Ir_mean * self._Ib_mean
        # Igg_var = cv2.blur(Ig * Ig, (r, r)) - self._Ig_mean * self._Ig_mean + eps
        # Igb_var = cv2.blur(Ig * Ib, (r, r)) - self._Ig_mean * self._Ib_mean
        # Ibb_var = cv2.blur(Ib * Ib, (r, r)) - self._Ib_mean * self._Ib_mean + eps


        self._Ir_mean = cv2.blur(Ir, (r, r))
        self._Ig_mean = cv2.blur(Ig, (r, r))
        self._Ib_mean = cv2.blur(Ib, (r, r))

        Irr_var = cv2.blur(Ir ** 2, (r, r)) - self._Ir_mean ** 2 + eps                                       
        Irg_var = cv2.blur(Ir * Ig, (r, r)) - self._Ir_mean * self._Ig_mean                                  
        Irb_var = cv2.blur(Ir * Ib, (r, r)) - self._Ir_mean * self._Ib_mean                                  
        Igg_var = cv2.blur(Ig * Ig, (r, r)) - self._Ig_mean * self._Ig_mean + eps                            
        Igb_var = cv2.blur(Ig * Ib, (r, r)) - self._Ig_mean * self._Ib_mean                                  
        Ibb_var = cv2.blur(Ib * Ib, (r, r)) - self._Ib_mean * self._Ib_mean + eps                                                       


        Irr_inv = Igg_var * Ibb_var - Igb_var * Igb_var                                                      
        Irg_inv = Igb_var * Irb_var - Irg_var * Ibb_var                                                      
        Irb_inv = Irg_var * Igb_var - Igg_var * Irb_var                                                      
        Igg_inv = Irr_var * Ibb_var - Irb_var * Irb_var                                                      
        Igb_inv = Irb_var * Irg_var - Irr_var * Igb_var                                                      
        Ibb_inv = Irr_var * Igg_var - Irg_var * Irg_var                                                      
        
        I_cov = Irr_inv * Irr_var + Irg_inv * Irg_var + Irb_inv * Irb_var                                    
        Irr_inv /= I_cov                                                                                     
        Irg_inv /= I_cov                                                                                     
        Irb_inv /= I_cov                                                                                     
        Igg_inv /= I_cov                                                                                     
        Igb_inv /= I_cov                                                                                     
        Ibb_inv /= I_cov                                                                                     
        
        self._Irr_inv = Irr_inv                                                                              
        self._Irg_inv = Irg_inv                                                                              
        self._Irb_inv = Irb_inv                                                                              
        self._Igg_inv = Igg_inv                                                                              
        self._Igb_inv = Igb_inv                                                                              
        self._Ibb_inv = Ibb_inv                  

    def _computeCoefficients(self, p):
        r = self._radius                                                             
        I = self._I                                                                 
        Ir, Ig, Ib = I[:, :, 0], I[:, :, 1], I[:, :, 2]                                                          
        

        p_mean = cv2.blur(p, (r, r))                             
        Ipr_mean = cv2.blur(Ir * p, (r, r))                                                         
        Ipg_mean = cv2.blur(Ig * p, (r, r))                                                    
        Ipb_mean = cv2.blur(Ib * p, (r, r))             



        Ipr_cov = Ipr_mean - self._Ir_mean * p_mean                                                 
        Ipg_cov = Ipg_mean - self._Ig_mean * p_mean                                                     
        Ipb_cov = Ipb_mean - self._Ib_mean * p_mean                                                       
                                                                                                                 
        ar = self._Irr_inv * Ipr_cov + self._Irg_inv * Ipg_cov + self._Irb_inv * Ipb_cov                 
        ag = self._Irg_inv * Ipr_cov + self._Igg_inv * Ipg_cov + self._Igb_inv * Ipb_cov                
        ab = self._Irb_inv * Ipr_cov + self._Igb_inv * Ipg_cov + self._Ibb_inv * Ipb_cov    

        b = p_mean - ar * self._Ir_mean - ag * self._Ig_mean - ab * self._Ib_mean                                                                                                                                         

        ar_mean = cv2.blur(ar, (r, r))          
        ag_mean = cv2.blur(ag, (r, r))                                                                   
        ab_mean = cv2.blur(ab, (r, r))                                                                      
        b_mean = cv2.blur(b, (r, r))                                                                                                                                              

        return ar_mean, ag_mean, ab_mean, b_mean            

    def _computeOutput(self, ab, I):
    
        ar_mean, ag_mean, ab_mean, b_mean = ab
        Ir, Ig, Ib = I[:, :, 0], I[:, :, 1], I[:, :, 2]
        q = ar_mean * Ir + ag_mean * Ig + ab_mean * Ib + b_mean
        return q

    def filter(self, p):

        p_32F = self._toFloatImg(p)

        ab = self._computeCoefficients(p)
        return self._computeOutput(ab, self._I)


In [5]:
# Guided Filter_He

from itertools import combinations_with_replacement
from collections import defaultdict

import numpy as np
from numpy.linalg import inv

R, G, B = 0, 1, 2  # index for convenience


def boxfilter(I, r):
    """Fast box filter implementation.
    Parameters
    ----------
    I:  a single channel/gray image data normalized to [0.0, 1.0]
    r:  window radius
    Return
    -----------
    The filtered image data.
    """
    M, N = I.shape
    dest = np.zeros((M, N))

    # cumulative sum over Y axis
    sumY = np.cumsum(I, axis=0)
    # difference over Y axis
    dest[:r + 1] = sumY[r: 2 * r + 1]
    dest[r + 1:M - r] = sumY[2 * r + 1:] - sumY[:M - 2 * r - 1]
    dest[-r:] = np.tile(sumY[-1], (r, 1)) - sumY[M - 2 * r - 1:M - r - 1]

    # cumulative sum over X axis
    sumX = np.cumsum(dest, axis=1)
    # difference over Y axis
    dest[:, :r + 1] = sumX[:, r:2 * r + 1]
    dest[:, r + 1:N - r] = sumX[:, 2 * r + 1:] - sumX[:, :N - 2 * r - 1]
    dest[:, -r:] = np.tile(sumX[:, -1][:, None], (1, r)) - \
        sumX[:, N - 2 * r - 1:N - r - 1]

    return dest


def guided_filter_he(I, p, r=40, eps=1e-3):
    """Refine a filter under the guidance of another (RGB) image.
    Parameters
    -----------
    I:   an M * N * 3 RGB image for guidance.
    p:   the M * N filter to be guided
    r:   the radius of the guidance
    eps: epsilon for the guided filter
    Return
    -----------
    The guided filter.
    """
    M, N = p.shape
    base = boxfilter(np.ones((M, N)), r)

    # each channel of I filtered with the mean filter
    means = [boxfilter(I[:, :, i], r) / base for i in range(3)]
    # p filtered with the mean filter
    mean_p = boxfilter(p, r) / base
    # filter I with p then filter it with the mean filter
    means_IP = [boxfilter(I[:, :, i] * p, r) / base for i in range(3)]
    # covariance of (I, p) in each local patch
    covIP = [means_IP[i] - means[i] * mean_p for i in range(3)]

    # variance of I in each local patch: the matrix Sigma in ECCV10 eq.14
    var = defaultdict(dict)
    for i, j in combinations_with_replacement(range(3), 2):
        var[i][j] = boxfilter(
            I[:, :, i] * I[:, :, j], r) / base - means[i] * means[j]

    a = np.zeros((M, N, 3))
    for y, x in np.ndindex(M, N):
        #         rr, rg, rb
        # Sigma = rg, gg, gb
        #         rb, gb, bb
        Sigma = np.array([[var[R][R][y, x], var[R][G][y, x], var[R][B][y, x]],
                          [var[R][G][y, x], var[G][G][y, x], var[G][B][y, x]],
                          [var[R][B][y, x], var[G][B][y, x], var[B][B][y, x]]])
        cov = np.array([c[y, x] for c in covIP])
        a[y, x] = np.dot(cov, inv(Sigma + eps * np.eye(3)))  # eq 14

    # ECCV10 eq.15
    b = mean_p - a[:, :, R] * means[R] - \
        a[:, :, G] * means[G] - a[:, :, B] * means[B]

    # ECCV10 eq.16
    q = (boxfilter(a[:, :, R], r) * I[:, :, R] + boxfilter(a[:, :, G], r) *
         I[:, :, G] + boxfilter(a[:, :, B], r) * I[:, :, B] + boxfilter(b, r)) / base

    return q



In [6]:
# Get Adaptive Exposure Map

np.seterr(over='ignore')
np.seterr(invalid ='ignore')
np.seterr(all ='ignore')


#
#  def AdaptiveExposureMap(img,sceneRadiance,Lambda,blockSize):
#     img = np.float32(img)
#     sceneRadiance = np.float32(sceneRadiance)
#
#     x = sceneRadiance * img + Lambda * (img **2 )
#     y = sceneRadiance ** 2 + Lambda *  (img ** 2)
#     S_x  = x / y
#
#     return S_x



def AdaptiveExposureMap(img, sceneRadiance, Lambda, blockSize):

    minValue = 10 ** -2
    img = np.uint8(img)
    sceneRadiance = np.uint8(sceneRadiance)

    YjCrCb = cv2.cvtColor(sceneRadiance, cv2.COLOR_BGR2YCrCb)
    YiCrCb = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
    normYjCrCb = (YjCrCb - YjCrCb.min()) / (YjCrCb.max() - YjCrCb.min())
    normYiCrCb = (YiCrCb - YiCrCb.min()) / (YiCrCb.max() - YiCrCb.min())
    Yi = normYiCrCb[:, :, 0]
    Yj = normYjCrCb[:, :, 0]
    Yi = np.clip(Yi, minValue,1)
    Yj = np.clip(Yj, minValue,1)

    # print('np.min(Yi)',np.min(Yi))
    # print('np.max(Yi)',np.max(Yi))
    # print('np.min(Yj)',np.min(Yj))
    # print('np.max(Yj)',np.max(Yj))
    # Yi = YiCrCb[:, :, 0]
    # Yj = YjCrCb[:, :, 0]
    S = (Yj * Yi + 0.3 * Yi ** 2) / (Yj ** 2 + 0.3 * Yi ** 2)

    # print('S',S)

    gimfiltR = 50  
    eps = 10 ** -3  

    # refinedS = guided_filter_he(YiCrCb, S, gimfiltR, eps)

    guided_filter = GuidedFilter(YiCrCb, gimfiltR, eps)
    # guided_filter = GuidedFilter(normYiCrCb, gimfiltR, eps)

    refinedS = guided_filter.filter(S)

    # print('guided_filter_he(YiCrCb, S, gimfiltR, eps)', refinedS)
    # S = np.clip(S, 0, 1)

    # cv2.imwrite('OutputImages_D/' + 'SSSSS' + '_GBdehazingRCorrectionStretching.jpg', np.uint8(S * 255))

    S_three = np.zeros(img.shape)
    S_three[:, :, 0] = S_three[:, :, 1] = S_three[:, :, 2] = refinedS

    return S_three

In [7]:
# Get Adaptive Scene Radiance

def AdaptiveSceneRadiance(sceneRadiance,S_x):
    sceneRadiance = np.float64(sceneRadiance)
    sceneRadiance  = sceneRadiance * S_x

    sceneRadiance = np.clip(sceneRadiance, 0, 255)
    sceneRadiance = np.uint8(sceneRadiance)
    return sceneRadiance

In [8]:
# Adaptive Atmospheric Light

class Node(object):
    def __init__(self,x,y,value):
        self.x = x
        self.y = y
        self.value = value
    def printInfo(self):
        print(self.x,self.y,self.value)


def getAtomsphericLight(darkChannel, img):
    img = np.float32(img)
    height = darkChannel.shape[0]
    width = darkChannel.shape[1]
    nodes = []
    
    for i in range(0, height):
        for j in range(0, width):
            oneNode = Node(i, j, darkChannel[i, j])
            nodes.append(oneNode)
    
    nodes = sorted(nodes, key=lambda node: node.value, reverse=False)
    # print('nodes[0]',nodes[0].value)
    # print('img[nodes[0].x, nodes[0].y, 0]+img[nodes[0].x, nodes[0].y, 1])',img[nodes[0].x, nodes[0].y, 0],img[nodes[0].x, nodes[0].y, 1])
    atomsphericLight  = np.mean([img[nodes[0].x, nodes[0].y, 0],img[nodes[0].x, nodes[0].y, 1]])
    atomsphericLightGB = img[nodes[0].x, nodes[0].y, 0:2]
    atomsphericLightRGB = img[nodes[0].x, nodes[0].y, :]
    # atomsphericLight  =  [img[nodes[0].x, nodes[0].y, 0],img[nodes[0].x, nodes[0].y, 1]]
    return atomsphericLight,atomsphericLightGB,atomsphericLightRGB

In [9]:
# Refined Transmission

def  refinedtransmission(transmission, img):
    gimfiltR = 50  
    eps = 10 ** -3  

    guided_filter = GuidedFilter(img, gimfiltR, eps)
    transmission[:,:,0] = guided_filter.filter(transmission[:,:,0])
    transmission[:,:,1] = guided_filter.filter(transmission[:,:,1])
    transmission = np.clip(transmission, 0.1, 0.9)


    return transmission

In [10]:
# Scene RadianceGb

def sceneRadianceGB(img,transmission,AtomsphericLight):
    sceneRadiance = img.copy()
    img = np.float32(img)
    for i in range(0, 2):
        sceneRadiance[:, :, i] = (img[:, :, i] - AtomsphericLight[i]) / transmission[:, :, i] + AtomsphericLight[i]
        
    sceneRadiance = (sceneRadiance - sceneRadiance.min()) / (sceneRadiance.max() - sceneRadiance.min()) * 255
    sceneRadiance = np.clip(sceneRadiance, 0, 255)
    sceneRadiance = np.uint8(sceneRadiance)

    return sceneRadiance


In [11]:
# Scene RadianceR

def  sceneradiance(img, sceneRadiance_GB):
    img = np.float32(img)
    R_original = img[:, :, 2]
    sceneRadiance_GB = np.float32(sceneRadiance_GB)

    print('***********************************************************')
    avgRr = 1.5 - (np.mean(sceneRadiance_GB[:,:,0])/255 +np.mean(sceneRadiance_GB[:,:,1])/255)
    parameterR  =   avgRr  / ((np.mean(R_original))/255)

    print('parameterR',parameterR)
    sceneRadianceR = R_original * parameterR
    sceneRadianceR = (sceneRadianceR - sceneRadianceR.min()) / (sceneRadianceR.max() - sceneRadianceR.min())
    sceneRadianceR = sceneRadianceR * 255

    sceneRadianceR = np.clip(sceneRadianceR, 0, 255)
    sceneRadiance_GB[:, :, 2] = sceneRadianceR
    sceneRadiance_GB = np.uint8(sceneRadiance_GB)
    return  sceneRadiance_GB

In [12]:
# Main

np.seterr(over='ignore')
if __name__ == '__main__':
    pass

starttime = datetime.datetime.now()

# folder = "C:/Users/Administrator/Desktop/UnderwaterImageEnhancement/Physical/GBdehazingRCorrection"
folder = r"C:\Users\Saptarshi\Project\Restore"
path = folder + "/InputImages"
files = os.listdir(path)
files =  natsort.natsorted(files)

for i in range(len(files)):
    file = files[i]
    filepath = path + "/" + file
    prefix = file.split('.')[0]
    if os.path.isfile(filepath):
        print('********    file   ********',file)
        img = cv2.imread(folder +'/InputImages/' + file)
        img = (img - img.min()) / (img.max() - img.min()) * 255
        blockSize = 9
        largestDiff = determineDepth(img, blockSize)
        AtomsphericLight, AtomsphericLightGB, AtomsphericLightRGB = getAtomsphericLight(largestDiff, img)
        print('AtomsphericLightRGB',AtomsphericLightRGB)
        
        # transmission = getTransmission(img, AtomsphericLightRGB, blockSize=blockSize)
        transmission = getTransmission(img, AtomsphericLightRGB, blockSize)
        # print('transmission.shape',transmission.shape)
        # TransmissionComposition(folder, transmission, number, param='coarse')
        transmission = refinedtransmission(transmission, img)

        cv2.imwrite('OutputImages/' + prefix + '_GBDehazedRcoorectionUDCP_TM.jpg', np.uint8(transmission[:, :, 0] * 255))


        # TransmissionComposition(folder, transmission, number, param='refined_15_175_175')
        sceneRadiance_GB = sceneRadianceGB(img, transmission, AtomsphericLightRGB)

        # cv2.imwrite('OutputImages/' + prefix + 'GBDehazed.jpg', sceneRadiance_GB)


        # # print('sceneRadiance_GB',sceneRadiance_GB)
        sceneRadiance = sceneradiance(img, sceneRadiance_GB)
        # sceneRadiance= sceneRadiance_GB
        # cv2.imwrite('OutputImages/'+ prefix + 'GBDehazedRcoorectionUDCP.jpg', sceneRadiance)
        # # print('np.min(sceneRadiance)',np.min(sceneRadiance))
        # # print('sceneRadiance',sceneRadiance)
        
        S_x = AdaptiveExposureMap(img, sceneRadiance, Lambda=0.3, blockSize=blockSize)
        # print('S_x',S_x)
        sceneRadiance = AdaptiveSceneRadiance(sceneRadiance, S_x)
        
        # print('sceneRadiance',sceneRadiance)
        cv2.imwrite('OutputImages/' + prefix + 'GBDehazedRcoorectionUDCPAdaptiveMap.jpg', sceneRadiance)


Endtime = datetime.datetime.now()
Time = Endtime - starttime
print('Time', Time)



********    file   ******** 2.png
AtomsphericLightRGB [162.40224 131.06145  91.17319]
***********************************************************
parameterR 2.213047216117398
********    file   ******** 5.png
AtomsphericLightRGB [183.88446  241.79283    9.143426]
***********************************************************
parameterR 9.570121974010885
********    file   ******** 34.png
AtomsphericLightRGB [255. 161.  40.]
***********************************************************
parameterR 8.161306321288125
********    file   ******** 52.png
AtomsphericLightRGB [238. 142.  32.]
***********************************************************
parameterR 8.694288795100471
********    file   ******** 102.png
AtomsphericLightRGB [255. 157.  39.]
***********************************************************
parameterR 8.721260048759438
********    file   ******** 358_img_.png
AtomsphericLightRGB [157.91284   86.55963   28.073395]
***********************************************************
paramet

********    file   ******** 12753.png
AtomsphericLightRGB [244. 175.   6.]
***********************************************************
parameterR 22.219149427635806
********    file   ******** 12960.png
AtomsphericLightRGB [198.73239  155.6338    44.295776]
***********************************************************
parameterR 5.707145057909679
********    file   ******** 12972.png
AtomsphericLightRGB [144. 113.  20.]
***********************************************************
parameterR 5.352839120602233
********    file   ******** 13252.png
AtomsphericLightRGB [255.        227.3494      1.0240964]
***********************************************************
parameterR 19.25450099777634
********    file   ******** 13259.png
AtomsphericLightRGB [173. 141.  90.]
***********************************************************
parameterR 7.257326777041773
********    file   ******** 15735.png
AtomsphericLightRGB [ 52. 208. 203.]
***********************************************************
param